# A/B Test: 광고 문구 생성 비교

## 목적
- **대조군 A**: 일반적인 OpenAI API를 활용해 단순히 생성한 광고 문구
- **실험군 B**: 업종별 분류(21개 하위 그룹)를 통해 특화된 광고 문구

## 가설
업종별 특화 광고 문구 시스템(B)이 일반 광고 문구 생성(A)보다 더 업종 특성에 맞고 효과적인 광고 문구를 생성한다.

## 테스트 구성
- 21개 하위 그룹별 10개의 한글 예시 문장
- 총 210개의 테스트 케이스
- 4개의 셀로 구성: S등급, A등급, B+C등급, D+E등급

## 0. 환경 설정 및 모듈 로드

In [1]:
import os
import sys
import json
from typing import Dict, List
from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 루트 경로 설정
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# 환경변수 로드
load_dotenv()

# 실험군 B용 모듈 import
from src.generation.text_generation import TextGenerator, PromptTemplateManager

print(f"프로젝트 경로: {PROJECT_ROOT}")
print("모듈 로드 완료!")

프로젝트 경로: c:\Users\USER\_codeit\_final_project\codeit_ad_smallbiz
모듈 로드 완료!


## 1. A/B 테스트 함수 정의

In [2]:
# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 실험군 B용 텍스트 생성기 초기화
text_generator_b = TextGenerator(use_industry_config=True)


def generate_ad_copy_control_a(korean_input: str, max_length: int = 20) -> str:
    """
    대조군 A: 일반적인 OpenAI API를 사용한 광고 문구 생성
    
    특징:
    - 업종 분류 없이 단순 광고 문구 생성
    - 일반적인 광고 카피라이팅 지시문 사용
    - 실험군 B와 비슷한 길이로 생성
    """
    system_prompt = f"""당신은 광고 카피라이터입니다. 주어진 내용을 바탕으로 광고 문구를 작성하세요.

규칙:
1. {max_length}자 이내로 작성 (공백 포함)
2. 짧고 임팩트 있는 문구 작성
3. 광고 문구만 출력 (설명, 번호 등 제외)
4. 한국어로 작성
5. 특수문자, 이모지 사용 금지
"""
    
    user_prompt = f"""다음 내용으로 광고 문구를 만들어주세요:

{korean_input}

광고 문구:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
            max_tokens=100
        )
        
        result = response.choices[0].message.content.strip()
        
        # 후처리
        result = result.replace('"', '').replace("'", "").replace('「', '').replace('」', '')
        result = result.replace("1. ", "").replace("- ", "")
        
        if len(result) > max_length:
            result = result[:max_length].strip()
        
        return result
        
    except Exception as e:
        return f"[Error] {str(e)}"


def generate_ad_copy_experiment_b(korean_input: str, max_length: int = 20) -> str:
    """
    실험군 B: 업종별 분류를 통한 특화 광고 문구 생성
    
    특징:
    - 한글 입력에서 자동 업종 감지
    - 18개 하위 그룹별 최적화된 톤앤매너 적용
    - 업종별 키워드, CTA, 타겟 고객 반영
    - AIDA 프레임워크 적용
    """
    try:
        result = text_generator_b.generate_ad_copy(
            user_input=korean_input,
            tone="auto",  # 업종에 맞는 톤 자동 선택
            max_length=max_length
        )
        return result
    except Exception as e:
        return f"[Error] {str(e)}"


def run_ab_test(test_cases: List[Dict], group_name: str):
    """
    A/B 테스트 실행 및 결과 출력
    
    Args:
        test_cases: 테스트 케이스 리스트 [{"group": "S-1", "input": "한글 입력"}]
        group_name: 그룹명 (예: "S등급")
    """
    print(f"\n{'='*100}")
    print(f"📊 {group_name} A/B 테스트 결과")
    print(f"{'='*100}\n")
    
    results = []
    
    for i, case in enumerate(test_cases, 1):
        group = case.get("group", "")
        korean_input = case.get("input", "")
        
        print(f"\n[{i}/{len(test_cases)}] {group}")
        print(f"-" * 80)
        
        # A/B 광고 문구 생성
        ad_copy_a = generate_ad_copy_control_a(korean_input)
        ad_copy_b = generate_ad_copy_experiment_b(korean_input)
        
        # 결과 출력
        print(f'한글 입력: "{korean_input}"')
        print(f'대조군 A: "{ad_copy_a}"')
        print(f'실험군 B: "{ad_copy_b}"')
        print(f"📏 길이 비교: A={len(ad_copy_a)}자, B={len(ad_copy_b)}자")
        
        results.append({
            "group": group,
            "korean_input": korean_input,
            "ad_copy_a": ad_copy_a,
            "ad_copy_b": ad_copy_b,
            "len_a": len(ad_copy_a),
            "len_b": len(ad_copy_b)
        })
    
    # 요약 통계
    avg_len_a = sum(r["len_a"] for r in results) / len(results)
    avg_len_b = sum(r["len_b"] for r in results) / len(results)
    
    print(f"\n\n{'='*100}")
    print(f"📈 {group_name} 요약 통계")
    print(f"{'='*100}")
    print(f"테스트 케이스 수: {len(results)}")
    print(f"대조군 A 평균 길이: {avg_len_a:.1f}자")
    print(f"실험군 B 평균 길이: {avg_len_b:.1f}자")
    
    return results

## 2. 테스트 데이터 정의

### 21개 하위 그룹별 10개의 한글 예시 문장

In [3]:
# ============================================
# S등급: 감성 및 미각 자극 그룹 (6개 그룹)
# ============================================

# S-1: 고온 조리 및 육즙 강조 (17개 업종)
S1_HOT_COOKING = [
    {"group": "S-1", "input": "삼겹살 맛집 홍보, 숯불 향 가득한 특선 고기"},
    {"group": "S-1", "input": "치킨 전문점, 바삭하고 촉촉한 후라이드 치킨"},
    {"group": "S-1", "input": "피자 가게 신메뉴, 쭉 늘어나는 치즈 피자"},
    {"group": "S-1", "input": "마라탕 전문점, 얼얼한 매운맛의 정석"},
    {"group": "S-1", "input": "스테이크 레스토랑, 입에서 녹는 프리미엄 안심"},
    {"group": "S-1", "input": "중식당 짜장면, 윤기 흐르는 면발의 유혹"},
    {"group": "S-1", "input": "버거 전문점, 육즙 터지는 수제 버거"},
    {"group": "S-1", "input": "곱창 구이, 불맛 가득 쫄깃한 곱창"},
    {"group": "S-1", "input": "라멘 전문점, 진한 돈코츠 국물의 깊은 맛"},
    {"group": "S-1", "input": "쌀국수 전문점, 향긋한 허브와 뜨끈한 국물"},
]

# S-2: 신선도 및 색감 강조 (8개 업종)
S2_FRESHNESS = [
    {"group": "S-2", "input": "횟집 홍보, 싱싱한 자연산 모둠회"},
    {"group": "S-2", "input": "초밥 전문점, 장인의 손길 프리미엄 오마카세"},
    {"group": "S-2", "input": "해산물 찜, 푸짐한 해물 한 상"},
    {"group": "S-2", "input": "수산시장, 산지직송 싱싱한 해산물"},
    {"group": "S-2", "input": "복어 요리, 담백하고 고급스러운 복코스"},
    {"group": "S-2", "input": "정육점 홍보, 마블링 최고급 한우"},
    {"group": "S-2", "input": "과일 전문점, 달콤한 제철 과일 선물세트"},
    {"group": "S-2", "input": "채소 가게, 농장직송 유기농 채소"},
    {"group": "S-2", "input": "건어물 전문점, 전통 방식 프리미엄 젓갈"},
    {"group": "S-2", "input": "연어 전문점, 신선한 노르웨이산 생연어"},
]

# S-3: 감성 및 부드러운 질감 (11개 업종)
S3_EMOTIONAL = [
    {"group": "S-3", "input": "카페 신메뉴, 달콤한 시즌 한정 음료"},
    {"group": "S-3", "input": "베이커리, 매일 갓 구운 수제 빵"},
    {"group": "S-3", "input": "아이스크림 전문점, 부드러운 프리미엄 젤라또"},
    {"group": "S-3", "input": "아이스크림 할인점, 인기 아이스크림 반값 세일"},
    {"group": "S-3", "input": "떡/한과 전문점, 정성 가득 전통 한과"},
    {"group": "S-3", "input": "샌드위치 전문점, 신선한 재료로 만든 건강식"},
    {"group": "S-3", "input": "슈퍼마켓, 신선식품 할인 대전"},
    {"group": "S-3", "input": "편의점 간편식, 간편하게 즐기는 한 끼"},
    {"group": "S-3", "input": "생수/음료 배달, 시원한 음료 정기 배송"},
    {"group": "S-3", "input": "우유 배달, 신선한 목장 우유 아침 배달"},
]

# S-4: 정갈한 한식과 상차림 (9개 업종)
S4_KOREAN_HOMESTYLE = [
    {"group": "S-4", "input": "한정식, 정갈한 반찬의 건강한 한 상"},
    {"group": "S-4", "input": "국밥집, 뜨끈한 국물로 속이 든든"},
    {"group": "S-4", "input": "족발/보쌈 전문점, 쫄깃 담백한 족보 세트"},
    {"group": "S-4", "input": "전 전문점, 바삭한 전과 막걸리 한 잔"},
    {"group": "S-4", "input": "칼국수 전문점, 쫄깃한 손칼국수"},
    {"group": "S-4", "input": "냉면 전문점, 시원한 한 그릇의 행복"},
    {"group": "S-4", "input": "기타 한식, 엄마 손맛 집밥 한 상"},
    {"group": "S-4", "input": "김밥/분식, 추억의 맛 분식 세트"},
    {"group": "S-4", "input": "반찬 가게, 정성 담은 수제 반찬"},
    {"group": "S-4", "input": "백반집, 오늘의 든든한 점심 정식"},
]

# S-5: 글로벌 다채로움 (4개 업종)
S5_GLOBAL_VARIETY = [
    {"group": "S-5", "input": "기타 일식, 정통 일본 가정식 한 상"},
    {"group": "S-5", "input": "기타 서양식, 유럽풍 코스 요리"},
    {"group": "S-5", "input": "동남아 전문점, 이국적인 향신료의 매력"},
    {"group": "S-5", "input": "분류 안된 외국식, 세계 각국의 맛 한자리에"},
    {"group": "S-5", "input": "태국 음식점, 향긋한 동남아 정통 맛"},
    {"group": "S-5", "input": "인도 커리 전문, 진한 향신료의 깊은 맛"},
    {"group": "S-5", "input": "멕시칸 레스토랑, 화려한 타코 파티"},
    {"group": "S-5", "input": "지중해 레스토랑, 건강한 올리브유 요리"},
    {"group": "S-5", "input": "터키 케밥 전문, 회전구이 고기의 진수"},
    {"group": "S-5", "input": "퓨전 레스토랑, 동서양의 맛있는 만남"},
]

# S-6: 화려한 무드와 야간 감성 (4개 업종)
S6_NIGHTLIFE = [
    {"group": "S-6", "input": "생맥주 전문점, 시원한 생맥주 한 잔"},
    {"group": "S-6", "input": "요리주점, 맛있는 안주와 함께하는 밤"},
    {"group": "S-6", "input": "일반 유흥 주점, 분위기 좋은 칵테일 바"},
    {"group": "S-6", "input": "무도 유흥 주점, 흥겨운 밤의 파티"},
    {"group": "S-6", "input": "이자카야, 일본식 선술집의 정취"},
    {"group": "S-6", "input": "와인바, 분위기 있는 와인 한 잔"},
    {"group": "S-6", "input": "크래프트 비어펍, 수제 맥주의 다양한 맛"},
    {"group": "S-6", "input": "재즈바, 라이브 음악과 함께"},
    {"group": "S-6", "input": "루프탑 바, 야경과 함께하는 칵테일"},
    {"group": "S-6", "input": "포장마차, 추억의 노포 감성"},
]

# S등급 전체 테스트 케이스
S_GRADE_TEST_CASES = S1_HOT_COOKING + S2_FRESHNESS + S3_EMOTIONAL + S4_KOREAN_HOMESTYLE + S5_GLOBAL_VARIETY + S6_NIGHTLIFE

print(f"S등급 테스트 케이스 수: {len(S_GRADE_TEST_CASES)}개")

S등급 테스트 케이스 수: 60개


In [4]:
# ============================================
# A등급: 비주얼 변화 및 자기관리 그룹 (4개 그룹)
# ============================================

# A-1: 인물 중심 뷰티 및 에스테틱 (6개 업종)
A1_BEAUTY = [
    {"group": "A-1", "input": "미용실 헤어 스타일링, 트렌디한 변신"},
    {"group": "A-1", "input": "피부관리실, 맑고 깨끗한 피부 만들기"},
    {"group": "A-1", "input": "피부/비뇨기과, 전문의 피부 진료"},
    {"group": "A-1", "input": "성형외과, 자연스러운 아름다움"},
    {"group": "A-1", "input": "네일샵, 손끝부터 시작되는 아름다움"},
    {"group": "A-1", "input": "화장품 매장, 나만의 뷰티 루틴"},
    {"group": "A-1", "input": "헤어샵 염색, 컬러로 완성하는 스타일"},
    {"group": "A-1", "input": "에스테틱, 프리미엄 피부 관리"},
    {"group": "A-1", "input": "속눈썹 연장, 또렷한 눈매 완성"},
    {"group": "A-1", "input": "왁싱샵, 매끈한 피부 관리"},
]

# A-2: 신체 변화 및 웰니스 (6개 업종)
A2_WELLNESS = [
    {"group": "A-2", "input": "헬스장 PT, 전문 트레이너와 함께하는 운동"},
    {"group": "A-2", "input": "요가/필라테스, 유연한 몸과 마음"},
    {"group": "A-2", "input": "태권도/무술학원, 건강한 심신 수련"},
    {"group": "A-2", "input": "종합 스포츠시설, 다양한 운동 한곳에서"},
    {"group": "A-2", "input": "체형/비만 관리, 건강한 다이어트"},
    {"group": "A-2", "input": "마사지/안마, 피로 회복 전문 케어"},
    {"group": "A-2", "input": "크로스핏, 고강도 체력 훈련"},
    {"group": "A-2", "input": "스피닝 센터, 신나는 음악과 함께 운동"},
    {"group": "A-2", "input": "복싱 체육관, 스트레스 해소 운동"},
    {"group": "A-2", "input": "댄스 학원, 즐거운 춤으로 건강하게"},
]

# A-3: 패션 및 스타일 아이템 (18개 업종)
A3_FASHION = [
    {"group": "A-3", "input": "여성 의류 매장, 트렌드 리딩 패션"},
    {"group": "A-3", "input": "남성 의류 매장, 깔끔한 스타일 완성"},
    {"group": "A-3", "input": "아동복 매장, 사랑스러운 키즈 패션"},
    {"group": "A-3", "input": "한복 전문점, 전통의 아름다움"},
    {"group": "A-3", "input": "기타 의류 매장, 나만의 스타일 찾기"},
    {"group": "A-3", "input": "신발 매장, 편안하고 스타일리시한 한 켤레"},
    {"group": "A-3", "input": "가방 매장, 매일 함께하는 패션 아이템"},
    {"group": "A-3", "input": "인테리어 디자인, 공간의 가치를 높이다"},
    {"group": "A-3", "input": "웨딩샵, 인생 최고의 순간을 위해"},
    {"group": "A-3", "input": "결혼 상담 서비스, 행복한 만남 주선"},
]

# A-4: 섬세한 케어 및 감성 소품 (11개 업종)
A4_DELICATE_CARE = [
    {"group": "A-4", "input": "꽃집, 마음을 전하는 꽃선물"},
    {"group": "A-4", "input": "반려동물 용품점, 반려동물 행복 쇼핑"},
    {"group": "A-4", "input": "동물병원, 반려동물 건강 지킴이"},
    {"group": "A-4", "input": "가구 매장, 편안한 공간 만들기"},
    {"group": "A-4", "input": "주방용품 매장, 요리가 즐거워지는 도구"},
    {"group": "A-4", "input": "건강보조식품, 건강한 일상을 위해"},
    {"group": "A-4", "input": "액세서리 매장, 포인트가 되는 소품"},
    {"group": "A-4", "input": "시계/귀금속, 시간을 담은 가치"},
    {"group": "A-4", "input": "안경 매장, 선명한 시야와 스타일"},
    {"group": "A-4", "input": "사진관, 소중한 순간을 영원히"},
]

# A등급 전체 테스트 케이스
A_GRADE_TEST_CASES = A1_BEAUTY + A2_WELLNESS + A3_FASHION + A4_DELICATE_CARE

print(f"A등급 테스트 케이스 수: {len(A_GRADE_TEST_CASES)}개")

A등급 테스트 케이스 수: 40개


In [5]:
# ============================================
# B등급: 공간 체험 및 교육 그룹 (3개 그룹)
# ============================================

# B-1: 학습 및 몰입 (18개 업종)
B1_LEARNING = [
    {"group": "B-1", "input": "입시학원, 명문대 합격의 지름길"},
    {"group": "B-1", "input": "외국어학원, 글로벌 인재 양성"},
    {"group": "B-1", "input": "전문자격 학원, 자격증 취득 전문"},
    {"group": "B-1", "input": "운전학원, 안전운전 1:1 교육"},
    {"group": "B-1", "input": "컴퓨터학원, IT 전문가 양성"},
    {"group": "B-1", "input": "독서실/스터디카페, 집중력 높은 학습 공간"},
    {"group": "B-1", "input": "음악학원, 예술적 감성 키우기"},
    {"group": "B-1", "input": "미술학원, 창의력을 깨우는 미술 교육"},
    {"group": "B-1", "input": "기타 교육기관, 평생학습의 시작"},
    {"group": "B-1", "input": "교육컨설팅, 맞춤형 교육 설계"},
]

# B-2: 여행 및 숙박 (9개 업종)
B2_LEISURE = [
    {"group": "B-2", "input": "호텔/리조트, 럭셔리한 휴식"},
    {"group": "B-2", "input": "여관/모텔, 편안한 비즈니스 숙박"},
    {"group": "B-2", "input": "펜션, 자연 속 프라이빗 휴가"},
    {"group": "B-2", "input": "캠핑/글램핑, 자연과 함께하는 힐링"},
    {"group": "B-2", "input": "기숙사/고시원, 합리적인 주거 공간"},
    {"group": "B-2", "input": "기타 숙박업, 편안한 여행의 시작"},
    {"group": "B-2", "input": "여행사, 완벽한 여행 플래닝"},
    {"group": "B-2", "input": "여행 예약 서비스, 쉽고 빠른 예약"},
    {"group": "B-2", "input": "목욕탕/사우나, 피로 회복 힐링"},
    {"group": "B-2", "input": "스파 리조트, 휴양과 건강 동시에"},
]

# B-3: 스포츠 및 활동 (19개 업종)
B3_SPORTS = [
    {"group": "B-3", "input": "수영장, 건강한 수영 라이프"},
    {"group": "B-3", "input": "볼링장, 즐거운 볼링 한 게임"},
    {"group": "B-3", "input": "당구장, 집중력을 키우는 당구"},
    {"group": "B-3", "input": "골프 연습장, 스윙 완성의 첫걸음"},
    {"group": "B-3", "input": "테니스장, 활기찬 테니스 레슨"},
    {"group": "B-3", "input": "탁구장, 건강한 탁구 동호회"},
    {"group": "B-3", "input": "노래방, 스트레스 해소 노래 한 곡"},
    {"group": "B-3", "input": "PC방, 최고 사양 게이밍 환경"},
    {"group": "B-3", "input": "전자 게임장, 신나는 아케이드"},
    {"group": "B-3", "input": "낚시터, 자연 속 힐링 낚시"},
]

# ============================================
# C등급: 신뢰 기반 전문 서비스 그룹 (3개 그룹)
# ============================================

# C-1: 전문 사무 (26개 업종)
C1_PROFESSIONAL = [
    {"group": "C-1", "input": "부동산 중개, 믿을 수 있는 매물 정보"},
    {"group": "C-1", "input": "변호사 사무소, 법률 전문가 상담"},
    {"group": "C-1", "input": "변리사 사무소, 특허/상표 등록 전문"},
    {"group": "C-1", "input": "세무사 사무소, 세금 걱정 해결"},
    {"group": "C-1", "input": "공인회계사, 재무 전문 컨설팅"},
    {"group": "C-1", "input": "광고 대행사, 효과적인 마케팅"},
    {"group": "C-1", "input": "경영 컨설팅, 사업 성장 파트너"},
    {"group": "C-1", "input": "번역/통역 서비스, 언어 장벽 해소"},
    {"group": "C-1", "input": "고용 알선, 맞춤 인력 매칭"},
    {"group": "C-1", "input": "전시/컨벤션 대행, 성공적인 행사"},
]

# C-2: 의료 및 케어 (18개 업종)
C2_MEDICAL = [
    {"group": "C-2", "input": "종합병원, 전문 의료진 진료"},
    {"group": "C-2", "input": "일반병원, 가까운 곳 건강 지킴이"},
    {"group": "C-2", "input": "치과병원, 건강한 치아 관리"},
    {"group": "C-2", "input": "한방병원, 한의학 전문 진료"},
    {"group": "C-2", "input": "내과/소아과, 가족 건강 주치의"},
    {"group": "C-2", "input": "안과 의원, 선명한 시력 관리"},
    {"group": "C-2", "input": "이비인후과, 귀코목 전문 진료"},
    {"group": "C-2", "input": "치과의원, 편안한 치료 경험"},
    {"group": "C-2", "input": "한의원, 체질에 맞는 한방 치료"},
    {"group": "C-2", "input": "약국, 건강 상담과 조제"},
]

# C-3: 기술 및 시공 (13개 업종)
C3_TECHNICAL = [
    {"group": "C-3", "input": "자동차 정비소, 믿을 수 있는 차량 관리"},
    {"group": "C-3", "input": "자동차 세차장, 깨끗한 내 차 관리"},
    {"group": "C-3", "input": "모터사이클 수리, 오토바이 전문 정비"},
    {"group": "C-3", "input": "건축 설계 서비스, 꿈의 공간 설계"},
    {"group": "C-3", "input": "조경 설계, 아름다운 정원 만들기"},
    {"group": "C-3", "input": "건물 청소 서비스, 깔끔한 공간 관리"},
    {"group": "C-3", "input": "소독/방제 서비스, 쾌적한 환경 만들기"},
    {"group": "C-3", "input": "시설 유지관리, 건물 관리 전문"},
    {"group": "C-3", "input": "엔지니어링 서비스, 기술 솔루션 제공"},
    {"group": "C-3", "input": "조경 유지관리, 사계절 정원 관리"},
]

# B+C등급 전체 테스트 케이스
BC_GRADE_TEST_CASES = B1_LEARNING + B2_LEISURE + B3_SPORTS + C1_PROFESSIONAL + C2_MEDICAL + C3_TECHNICAL

print(f"B+C등급 테스트 케이스 수: {len(BC_GRADE_TEST_CASES)}개")

B+C등급 테스트 케이스 수: 60개


In [6]:
# ============================================
# D등급: 목적형 소매 및 장비 그룹 (3개 그룹)
# ============================================

# D-1: 중장비 및 부품 (13개 업종)
D1_HEAVY_EQUIPMENT = [
    {"group": "D-1", "input": "타이어 매장, 안전한 주행의 시작"},
    {"group": "D-1", "input": "자동차 부품 매장, 정품 부품 전문"},
    {"group": "D-1", "input": "모터사이클 부품, 오토바이 액세서리"},
    {"group": "D-1", "input": "철물/공구 매장, 든든한 작업 파트너"},
    {"group": "D-1", "input": "바닥재 매장, 마루/장판 전문 시공"},
    {"group": "D-1", "input": "건축자재 판매, 품질 좋은 자재 공급"},
    {"group": "D-1", "input": "전기용품 매장, LED 조명 전문"},
    {"group": "D-1", "input": "건설장비 대여, 필요할 때 바로 렌탈"},
    {"group": "D-1", "input": "산업용 기계 대여, 효율적인 장비 임대"},
    {"group": "D-1", "input": "가정용 연료 판매, 등유 배달 서비스"},
]

# D-2: 일반 기기 (14개 업종)
D2_GENERAL_EQUIPMENT = [
    {"group": "D-2", "input": "컴퓨터/소프트웨어, IT 전문 매장"},
    {"group": "D-2", "input": "핸드폰 매장, 최신 스마트폰 개통"},
    {"group": "D-2", "input": "가전제품 매장, 생활가전 할인"},
    {"group": "D-2", "input": "사무기기 판매, 오피스 솔루션"},
    {"group": "D-2", "input": "컴퓨터 수리, 빠른 AS 서비스"},
    {"group": "D-2", "input": "핸드폰 수리, 당일 액정 교체"},
    {"group": "D-2", "input": "가전 수리, 출장 서비스 전문"},
    {"group": "D-2", "input": "자동차 렌탈, 편리한 차량 이용"},
    {"group": "D-2", "input": "세탁소, 깔끔한 드라이클리닝"},
    {"group": "D-2", "input": "셀프 빨래방, 24시간 코인 세탁"},
]

# D-3: 취미 및 라이프스타일 소매 (18개 업종)
D3_HOBBY_LIFESTYLE = [
    {"group": "D-3", "input": "악기 매장, 음악의 시작"},
    {"group": "D-3", "input": "음반/비디오 매장, 음악과 영상의 세계"},
    {"group": "D-3", "input": "담배/전자담배 매장, 다양한 선택"},
    {"group": "D-3", "input": "주류 매장, 프리미엄 와인 셀렉션"},
    {"group": "D-3", "input": "서점, 책과 함께하는 시간"},
    {"group": "D-3", "input": "문구/회화용품, 창작의 도구"},
    {"group": "D-3", "input": "장난감 매장, 아이들의 꿈 가게"},
    {"group": "D-3", "input": "운동용품 매장, 스포츠 라이프"},
    {"group": "D-3", "input": "자전거 매장, 건강한 라이딩"},
    {"group": "D-3", "input": "기념품점, 여행의 추억"},
]

# ============================================
# E등급: 인프라 및 특수 목적 그룹 (2개 그룹)
# ============================================

# E-1: 편의 및 인프라 (3개 업종)
E1_INFRASTRUCTURE = [
    {"group": "E-1", "input": "주유소, 합리적인 주유 가격"},
    {"group": "E-1", "input": "가스 충전소, 안전한 가스 충전"},
    {"group": "E-1", "input": "얼음 소매, 대용량 얼음 배달"},
    {"group": "E-1", "input": "셀프 주유소, 24시간 편리한 주유"},
    {"group": "E-1", "input": "전기차 충전소, 친환경 충전"},
    {"group": "E-1", "input": "복합 주유소, 편의시설 완비"},
    {"group": "E-1", "input": "수소 충전소, 미래 에너지"},
    {"group": "E-1", "input": "휴게소 주유소, 편안한 휴식"},
    {"group": "E-1", "input": "야간 주유소, 심야 영업"},
    {"group": "E-1", "input": "카페 겸 주유소, 휴식과 함께"},
]

# E-2: 예절 및 의례 (2개 업종)
E2_CEREMONY = [
    {"group": "E-2", "input": "장례식장, 정성을 다하는 추모"},
    {"group": "E-2", "input": "화장터/납골당, 평화로운 안식처"},
    {"group": "E-2", "input": "상조 서비스, 든든한 장례 준비"},
    {"group": "E-2", "input": "수목장, 자연과 함께하는 추모"},
    {"group": "E-2", "input": "추모관, 영원한 기억의 공간"},
    {"group": "E-2", "input": "장례 화환, 마음을 담은 조화"},
    {"group": "E-2", "input": "묘지 관리, 정성스러운 관리"},
    {"group": "E-2", "input": "추모 공간, 차분한 위로"},
    {"group": "E-2", "input": "장지, 고요한 안식"},
    {"group": "E-2", "input": "예배당, 평화로운 영성"},
]

# D+E등급 전체 테스트 케이스
DE_GRADE_TEST_CASES = D1_HEAVY_EQUIPMENT + D2_GENERAL_EQUIPMENT + D3_HOBBY_LIFESTYLE + E1_INFRASTRUCTURE + E2_CEREMONY

print(f"D+E등급 테스트 케이스 수: {len(DE_GRADE_TEST_CASES)}개")

D+E등급 테스트 케이스 수: 50개


## 3. A/B 테스트 실행

### 3.1 S등급 테스트 (감성 및 미각 자극)

In [7]:
# S등급 A/B 테스트 실행
s_grade_results = run_ab_test(S_GRADE_TEST_CASES, "S등급 (감성 및 미각 자극)")


📊 S등급 (감성 및 미각 자극) A/B 테스트 결과


[1/60] S-1
--------------------------------------------------------------------------------
한글 입력: "삼겹살 맛집 홍보, 숯불 향 가득한 특선 고기"
대조군 A: "숯불 삼겹살의 향연"
실험군 B: "숯불향 삼겹살 지금 주문!"
📏 길이 비교: A=10자, B=14자

[2/60] S-1
--------------------------------------------------------------------------------
한글 입력: "치킨 전문점, 바삭하고 촉촉한 후라이드 치킨"
대조군 A: "바삭한 유혹, 치킨의 정석"
실험군 B: "바삭 촉촉, 지금 주문!"
📏 길이 비교: A=14자, B=13자

[3/60] S-1
--------------------------------------------------------------------------------
한글 입력: "피자 가게 신메뉴, 쭉 늘어나는 치즈 피자"
대조군 A: "치즈가 쭉 늘어나는 피자!"
실험군 B: "치즈가 쭉 늘어나는 피자, 지금 주문"
📏 길이 비교: A=14자, B=20자

[4/60] S-1
--------------------------------------------------------------------------------
한글 입력: "마라탕 전문점, 얼얼한 매운맛의 정석"
대조군 A: "매운맛의 정수를 느껴라"
실험군 B: "얼얼한 매운맛을 지금!"
📏 길이 비교: A=12자, B=12자

[5/60] S-1
--------------------------------------------------------------------------------
한글 입력: "스테이크 레스토랑, 입에서 녹는 프리미엄 안심"
대조군 A: "입에서 녹는 프리미엄 스테이크"
실험군 B: "입에서 녹는 안심 스테이크"
📏 길이 비교: 

### 3.2 A등급 테스트 (비주얼 변화 및 자기관리)

In [8]:
# A등급 A/B 테스트 실행
a_grade_results = run_ab_test(A_GRADE_TEST_CASES, "A등급 (비주얼 변화 및 자기관리)")


📊 A등급 (비주얼 변화 및 자기관리) A/B 테스트 결과


[1/40] A-1
--------------------------------------------------------------------------------
한글 입력: "미용실 헤어 스타일링, 트렌디한 변신"
대조군 A: "트렌디한 변신, 지금 시작하세요"
실험군 B: "변신의 시작, 맞춤 스타일링 예약하세"
📏 길이 비교: A=17자, B=20자

[2/40] A-1
--------------------------------------------------------------------------------
한글 입력: "피부관리실, 맑고 깨끗한 피부 만들기"
대조군 A: "깨끗한 피부, 당신의 자신감"
실험군 B: "나만의 맞춤 피부 변신!"
📏 길이 비교: A=15자, B=13자

[3/40] A-1
--------------------------------------------------------------------------------
한글 입력: "피부/비뇨기과, 전문의 피부 진료"
대조군 A: "전문의의 피부 케어"
실험군 B: "전문의와 함께하는 피부 케어"
📏 길이 비교: A=10자, B=15자

[4/40] A-1
--------------------------------------------------------------------------------
한글 입력: "성형외과, 자연스러운 아름다움"
대조군 A: "자연스러운 아름다움, 당신의 선택"
실험군 B: "자연스러운 아름다움, 맞춤형 상담 예"
📏 길이 비교: A=18자, B=20자

[5/40] A-1
--------------------------------------------------------------------------------
한글 입력: "네일샵, 손끝부터 시작되는 아름다움"
대조군 A: "손끝에서 피어나는 아름다움"
실험군 B: "손끝에서 시작되는 변신"
📏 길이 비교: A=14자,

### 3.3 B+C등급 테스트 (공간 체험/교육 + 신뢰 기반 서비스)

In [9]:
# B+C등급 A/B 테스트 실행
bc_grade_results = run_ab_test(BC_GRADE_TEST_CASES, "B+C등급 (공간 체험/교육 + 신뢰 기반 서비스)")


📊 B+C등급 (공간 체험/교육 + 신뢰 기반 서비스) A/B 테스트 결과


[1/60] B-1
--------------------------------------------------------------------------------
한글 입력: "입시학원, 명문대 합격의 지름길"
대조군 A: "명문대, 입시의 시작!"
실험군 B: "명문대 합격, 체계적 관리로!"
📏 길이 비교: A=12자, B=16자

[2/60] B-1
--------------------------------------------------------------------------------
한글 입력: "외국어학원, 글로벌 인재 양성"
대조군 A: "글로벌 인재, 여기서 시작!"
실험군 B: "글로벌 인재, 함께 성장해요"
📏 길이 비교: A=15자, B=15자

[3/60] B-1
--------------------------------------------------------------------------------
한글 입력: "전문자격 학원, 자격증 취득 전문"
대조군 A: "자격증 취득, 여기서 시작!"
실험군 B: "자격증, 체계적 관리로 취득하세요"
📏 길이 비교: A=15자, B=18자

[4/60] B-1
--------------------------------------------------------------------------------
한글 입력: "운전학원, 안전운전 1:1 교육"
대조군 A: "안전운전, 1:1로 배워요"
실험군 B: "안전운전, 1:1 전문 교육"
📏 길이 비교: A=14자, B=15자

[5/60] B-1
--------------------------------------------------------------------------------
한글 입력: "컴퓨터학원, IT 전문가 양성"
대조군 A: "IT 전문가의 첫걸음"
실험군 B: "IT 전문가의 꿈, 함께 키워요!"
📏 길이 비교: A=11자, B=18

### 3.4 D+E등급 테스트 (목적형 소매 + 인프라)

In [10]:
# D+E등급 A/B 테스트 실행
de_grade_results = run_ab_test(DE_GRADE_TEST_CASES, "D+E등급 (목적형 소매 + 인프라)")


📊 D+E등급 (목적형 소매 + 인프라) A/B 테스트 결과


[1/50] D-1
--------------------------------------------------------------------------------
한글 입력: "타이어 매장, 안전한 주행의 시작"
대조군 A: "안전한 주행, 여기서 시작!"
실험군 B: "안전한 주행, 믿고 선택하세요"
📏 길이 비교: A=15자, B=16자

[2/50] D-1
--------------------------------------------------------------------------------
한글 입력: "자동차 부품 매장, 정품 부품 전문"
대조군 A: "정품 부품, 안전한 선택"
실험군 B: "정품 부품, 신뢰의 선택"
📏 길이 비교: A=13자, B=13자

[3/50] D-1
--------------------------------------------------------------------------------
한글 입력: "모터사이클 부품, 오토바이 액세서리"
대조군 A: "모터사이클의 완벽한 동반자"
실험군 B: "모터사이클, 신나는 여정의 시작!"
📏 길이 비교: A=14자, B=18자

[4/50] D-1
--------------------------------------------------------------------------------
한글 입력: "철물/공구 매장, 든든한 작업 파트너"
대조군 A: "작업의 든든한 동반자"
실험군 B: "당일 출고, 저렴한 철물!"
📏 길이 비교: A=11자, B=14자

[5/50] D-1
--------------------------------------------------------------------------------
한글 입력: "바닥재 매장, 마루/장판 전문 시공"
대조군 A: "완벽한 바닥, 눈에 띄다"
실험군 B: "당일 설치, 저렴하게 바닥재!"
📏 길이 비교: A=13자, B=16자



## 4. 종합 분석 및 결과 저장

In [11]:
import pandas as pd
from datetime import datetime

# 모든 결과 합치기
all_results = s_grade_results + a_grade_results + bc_grade_results + de_grade_results

# DataFrame으로 변환
df = pd.DataFrame(all_results)

# 기본 통계
print("\n" + "="*100)
print("📊 전체 A/B 테스트 종합 결과")
print("="*100)

print(f"\n총 테스트 케이스: {len(all_results)}개")
print(f"\n대조군 A 평균 길이: {df['len_a'].mean():.1f}자")
print(f"실험군 B 평균 길이: {df['len_b'].mean():.1f}자")
print(f"\n대조군 A 길이 범위: {df['len_a'].min()} ~ {df['len_a'].max()}자")
print(f"실험군 B 길이 범위: {df['len_b'].min()} ~ {df['len_b'].max()}자")

# 그룹별 통계
print("\n" + "-"*50)
print("그룹별 평균 길이 비교")
print("-"*50)
group_stats = df.groupby('group').agg({
    'len_a': 'mean',
    'len_b': 'mean'
}).round(1)
group_stats.columns = ['대조군 A 평균', '실험군 B 평균']
print(group_stats)

# 결과 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"{timestamp}_ab_test_ad_copy_results.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 결과 저장 완료: {output_path}")


📊 전체 A/B 테스트 종합 결과

총 테스트 케이스: 210개

대조군 A 평균 길이: 13.1자
실험군 B 평균 길이: 15.7자

대조군 A 길이 범위: 8 ~ 19자
실험군 B 길이 범위: 10 ~ 20자

--------------------------------------------------
그룹별 평균 길이 비교
--------------------------------------------------
       대조군 A 평균  실험군 B 평균
group                    
A-1        14.1      15.7
A-2        12.9      17.5
A-3        13.3      16.3
A-4        13.1      15.4
B-1        13.0      15.0
B-2        12.9      17.7
B-3        13.0      15.2
C-1        13.8      15.4
C-2        13.5      16.8
C-3        13.8      17.6
D-1        12.6      15.6
D-2        13.4      16.1
D-3        13.1      15.1
E-1        12.3      16.3
E-2        11.6      13.3
S-1        12.4      15.7
S-2        12.6      13.5
S-3        13.5      15.8
S-4        12.9      15.4
S-5        12.3      15.5
S-6        14.0      14.4

💾 결과 저장 완료: 20260127_033626_ab_test_ad_copy_results.csv


## 5. GPT 기반 광고 문구 품질 평가

### 5.1 평가 기준 정의 (업종별 분류 문서 기반)

In [12]:
# ============================================
# GPT 기반 광고 문구 품질 평가 시스템
# ============================================

# 등급별 평가 기준 (업종별_분류_및_광고전략.md 기반)
EVALUATION_CRITERIA = {
    "S": {
        "name": "S등급 - 감성 및 미각 자극",
        "criteria": """
## S등급 광고 문구 평가 기준 (감성 및 미각 자극)

### 핵심 원칙
- 이성적인 판단보다 '시각적/감각적 자극'에 의한 즉각적 반응 유도
- 광고 문구 한 줄이 고객의 식욕과 감성을 자극해야 함

### 세부 평가 항목 (각 20점, 총 100점)

1. **감각적 표현 (Sensory Appeal)** - 20점
   - S-1: 육즙, 겉바속촉, 지글지글, 갓 구운 등 식욕 자극 표현
   - S-2: 신선함, 산지직송, 프리미엄, 투명한 등 고급감 표현
   - S-3: 달콤한, 부드러운, 촉촉한 등 감성적 표현
   - S-4: 정갈한, 엄마 손맛, 정성 등 가정적 표현

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - S-1: 20-40대 직장인, 회식/야식 상황
   - S-2: 30-50대, 특별한 날/접대 상황
   - S-3: 20-30대 여성, SNS 활동적
   - S-4: 30-60대, 바쁜 직장인/1인 가구

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - S-1: 친근하고 감각적, 식욕 자극
   - S-2: 신뢰감 있고 고급스러운
   - S-3: 따뜻하고 감성적, 위로하는
   - S-4: 따뜻하고 믿음직한

4. **CTA 명확성 (Call-to-Action)** - 20점
   - 고객이 취해야 할 행동이 명확한가
   - 지금 주문, 방문, 전화 등 구체적 행동 유도

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    },
    "A": {
        "name": "A등급 - 비주얼 변화 및 자기관리",
        "criteria": """
## A등급 광고 문구 평가 기준 (비주얼 변화 및 자기관리)

### 핵심 원칙
- '변화'와 '결과'를 언어로 표현하여 기대감 형성
- 전문성과 개인 맞춤 서비스 강조

### 세부 평가 항목 (각 20점, 총 100점)

1. **변화/결과 표현 (Transformation)** - 20점
   - A-1: 찰랑거리는 머릿결, 맑아진 피부, 아름다운 변화
   - A-2: 탄탄한 몸매, -10kg 성공, 건강한 변화
   - A-3: 나다운 스타일, 트렌디한, 세련된
   - A-4: 섬세한, 특별한, 유니크한

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - A-1: 20-40대 여성, 변화 원함
   - A-2: 25-45대, 건강/체력 목표
   - A-3: 20-30대, 트렌드+실용적 직장인
   - A-4: 20-40대, 선물/특별한 케어

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - A-1: 전문적이면서 친근한
   - A-2: 동기부여하고 격려하는
   - A-3: 친근하고 트렌디한
   - A-4: 섬세하고 친절한

4. **전문성/신뢰감 (Professionalism)** - 20점
   - 전문가, 경력, 맞춤 서비스 등 신뢰 요소
   - 1:1 상담, 개인 맞춤 등 차별화 요소

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    },
    "BC": {
        "name": "B+C등급 - 공간 체험/신뢰 기반 서비스",
        "criteria": """
## B+C등급 광고 문구 평가 기준 (공간 체험 + 신뢰 기반 서비스)

### B등급 핵심 원칙
- '머무는 경험'과 '성취'를 언어로 표현
- 시설의 품질과 환경 강조

### C등급 핵심 원칙
- '실패 없는 선택'임을 증명하는 신뢰성 강조
- 전문 자격, 경력, 정확한 정보 전달

### 세부 평가 항목 (각 20점, 총 100점)

1. **핵심 가치 전달 (Core Value)** - 20점
   - B-1: 집중, 성장, 쾌적함, 체계적 관리
   - B-2: 휴식, 탈출, 재충전, 힐링
   - B-3: 활력, 시설 품질, 실력 향상
   - C등급: 전문성, 신뢰, 안심

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - B-1: 학부모/성인 학습자
   - B-2: 30-50대 가족/커플
   - B-3: 30-60대 운동 관심층
   - C등급: 전문 서비스 필요 고객

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - B등급: 전문적/활기찬/따뜻한
   - C등급: 믿음직하고 전문적인

4. **신뢰 요소 (Trust Elements)** - 20점
   - 경력, 자격, 시설, 실적 등 객관적 신뢰 요소
   - 전문가, 합격률, A/S 보증 등

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    },
    "DE": {
        "name": "D+E등급 - 목적형 소매/인프라",
        "criteria": """
## D+E등급 광고 문구 평가 기준 (목적형 소매 + 인프라)

### D등급 핵심 원칙
- 견고함, 실용성, 기능성 강조
- 가격, 재고, 품질 정보 명확히 전달

### E등급 핵심 원칙
- 접근성, 편의성, 품위 강조
- 직관적이고 명확한 정보 전달

### 세부 평가 항목 (각 20점, 총 100점)

1. **핵심 정보 전달 (Key Information)** - 20점
   - D-1: 재고 보유, 당일 출고, 정품, 내구성
   - D-2: 최신 모델, 고사양, 할인, A/S
   - E-1: 위치, 가격, 24시간, 편의시설
   - E-2: 엄숙한, 정중한, 깨끗한, 세심한

2. **타겟 고객 적합성 (Target Fit)** - 20점
   - D-1: 30-60대 차주/공사 관계자
   - D-2: 20-50대 기기 구매 계획자
   - E-1: 모든 연령대 운전자
   - E-2: 장례 서비스 필요 고객

3. **브랜드 보이스 (Tone & Manner)** - 20점
   - D등급: 실용적이고 직관적인
   - E-1: 직관적이고 명확한
   - E-2: 차분하고 정중한

4. **실용성/혜택 (Practicality)** - 20점
   - 가격 혜택, 할인, 특가 등 실질적 이익
   - 편의성, 접근성 등 실용적 가치

5. **업종 특화 키워드 (Industry-Specific)** - 20점
   - 해당 업종에 특화된 전문 용어/키워드 포함 여부
"""
    }
}

print("평가 기준 로드 완료!")

평가 기준 로드 완료!


### 5.2 GPT 평가 함수 정의

In [13]:
def evaluate_ad_copy_with_gpt(
    korean_input: str,
    ad_copy_a: str,
    ad_copy_b: str,
    group: str
) -> Dict:
    """
    GPT를 사용하여 두 광고 문구를 비교 평가

    Args:
        korean_input: 원본 한글 입력
        ad_copy_a: 대조군 A 광고 문구
        ad_copy_b: 실험군 B 광고 문구
        group: 그룹 코드 (S-1, A-2 등)

    Returns:
        Dict: 평가 결과
    """
    # 등급 결정
    if group.startswith("S"):
        grade = "S"
    elif group.startswith("A"):
        grade = "A"
    elif group.startswith("B") or group.startswith("C"):
        grade = "BC"
    else:
        grade = "DE"

    criteria = EVALUATION_CRITERIA[grade]["criteria"]
    grade_name = EVALUATION_CRITERIA[grade]["name"]

    system_prompt = f"""당신은 소상공인 광고 문구 품질을 평가하는 전문가입니다.
광고 문구의 효과성과 업종 적합성을 평가합니다.

{criteria}

## 평가 방법
1. 각 항목별로 0-20점 척도로 평가
2. 두 광고 문구(A와 B)를 동일한 기준으로 평가
3. 어떤 광고 문구가 해당 업종에 더 효과적인지 판단
4. 반드시 JSON 형식으로 출력

## 출력 형식 (JSON)
{{
    "grade": "{grade_name}",
    "evaluation_a": {{
        "sensory_or_value": <0-20>,
        "target_fit": <0-20>,
        "tone_manner": <0-20>,
        "cta_or_trust": <0-20>,
        "industry_specific": <0-20>,
        "total": <0-100>,
        "strengths": "강점 설명",
        "weaknesses": "약점 설명"
    }},
    "evaluation_b": {{
        "sensory_or_value": <0-20>,
        "target_fit": <0-20>,
        "tone_manner": <0-20>,
        "cta_or_trust": <0-20>,
        "industry_specific": <0-20>,
        "total": <0-100>,
        "strengths": "강점 설명",
        "weaknesses": "약점 설명"
    }},
    "winner": "A" 또는 "B" 또는 "TIE",
    "winner_reason": "선택 이유 (2-3문장)",
    "detailed_comparison": "두 광고 문구의 상세 비교 분석 (3-4문장)"
}}
"""

    user_prompt = f"""## 평가 대상

### 원본 한글 입력 (광고 요청)
"{korean_input}"

### 업종 그룹
{group}

### 대조군 A 광고 문구
"{ad_copy_a}"

### 실험군 B 광고 문구
"{ad_copy_b}"

위 두 광고 문구를 평가 기준에 따라 비교 분석하고, 어떤 광고 문구가 해당 업종에 더 효과적인지 판단해주세요.
반드시 JSON 형식으로만 응답해주세요."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # 더 정확한 평가를 위해 gpt-4o 사용
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # 일관된 평가를 위해 낮은 temperature
            response_format={"type": "json_object"}
        )

        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        return {"error": str(e)}


def evaluate_existing_results(existing_results: List[Dict], group_name: str, sample_size: int = None):
    """
    이미 생성된 A/B 테스트 결과를 사용하여 GPT 평가만 수행
    
    Args:
        existing_results: 섹션 3에서 생성된 A/B 테스트 결과 리스트
        group_name: 그룹명
        sample_size: 평가할 샘플 수 (None이면 전체 평가)
    
    Returns:
        평가 결과 리스트, 승패 통계
    """
    import random
    import time
    
    print(f"\n{'='*100}")
    print(f"📊 {group_name} GPT 평가 (기존 A/B 결과 활용)")
    print(f"{'='*100}\n")
    
    # 샘플 선택 (None이면 전체)
    if sample_size and sample_size < len(existing_results):
        samples = random.sample(existing_results, sample_size)
        print(f"📌 전체 {len(existing_results)}개 중 {sample_size}개 샘플 평가")
    else:
        samples = existing_results
        print(f"📌 전체 {len(existing_results)}개 평가")
    
    results = []
    wins = {"A": 0, "B": 0, "TIE": 0}
    total_scores = {"A": [], "B": []}
    
    for i, case in enumerate(samples, 1):
        group = case.get("group", "")
        korean_input = case.get("korean_input", "")
        ad_copy_a = case.get("ad_copy_a", "")
        ad_copy_b = case.get("ad_copy_b", "")
        
        print(f"\n[{i}/{len(samples)}] {group}")
        print(f"-" * 80)
        print(f'한글 입력: "{korean_input}"')
        print(f'대조군 A: "{ad_copy_a}"')
        print(f'실험군 B: "{ad_copy_b}"')
        
        # GPT 평가
        print("\n🔍 GPT 평가 중...")
        evaluation = evaluate_ad_copy_with_gpt(korean_input, ad_copy_a, ad_copy_b, group)
        
        if "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            winner = evaluation.get("winner", "TIE")
            
            print(f"\n📊 평가 결과:")
            print(f"   대조군 A 점수: {eval_a.get('total', 'N/A')}점")
            print(f"   실험군 B 점수: {eval_b.get('total', 'N/A')}점")
            print(f"   🏆 승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")
            print(f"   📝 이유: {evaluation.get('winner_reason', 'N/A')}")
            
            wins[winner] += 1
            if eval_a.get('total'):
                total_scores["A"].append(eval_a.get('total'))
            if eval_b.get('total'):
                total_scores["B"].append(eval_b.get('total'))
            
            results.append({
                "group": group,
                "korean_input": korean_input,
                "ad_copy_a": ad_copy_a,
                "ad_copy_b": ad_copy_b,
                "len_a": case.get("len_a", len(ad_copy_a)),
                "len_b": case.get("len_b", len(ad_copy_b)),
                "score_a": eval_a.get('total'),
                "score_b": eval_b.get('total'),
                "winner": winner,
                "evaluation": evaluation
            })
        else:
            print(f"   ❌ 평가 오류: {evaluation['error']}")
            # 오류가 나도 결과에 포함 (평가 없이)
            results.append({
                "group": group,
                "korean_input": korean_input,
                "ad_copy_a": ad_copy_a,
                "ad_copy_b": ad_copy_b,
                "len_a": case.get("len_a", len(ad_copy_a)),
                "len_b": case.get("len_b", len(ad_copy_b)),
                "score_a": None,
                "score_b": None,
                "winner": None,
                "evaluation": evaluation
            })
        
        # API 레이트 리밋 방지를 위한 딜레이
        if i < len(samples):
            time.sleep(0.5)
    
    # 요약 통계
    valid_results = [r for r in results if r.get("winner")]
    
    print(f"\n\n{'='*100}")
    print(f"📈 {group_name} 평가 요약")
    print(f"{'='*100}")
    print(f"평가 케이스 수: {len(valid_results)}/{len(samples)}")
    
    if valid_results:
        print(f"\n🏆 승패 현황:")
        print(f"   대조군 A 승: {wins['A']}회 ({wins['A']/len(valid_results)*100:.1f}%)")
        print(f"   실험군 B 승: {wins['B']}회 ({wins['B']/len(valid_results)*100:.1f}%)")
        print(f"   무승부: {wins['TIE']}회 ({wins['TIE']/len(valid_results)*100:.1f}%)")
    
    if total_scores["A"] and total_scores["B"]:
        print(f"\n📊 평균 점수:")
        print(f"   대조군 A: {sum(total_scores['A'])/len(total_scores['A']):.1f}점")
        print(f"   실험군 B: {sum(total_scores['B'])/len(total_scores['B']):.1f}점")
    
    return results, wins


# 기존 run_ab_test_with_evaluation 함수는 유지 (필요시 사용)
def run_ab_test_with_evaluation(test_cases: List[Dict], group_name: str, sample_size: int = 5):
    """
    A/B 테스트 실행 + GPT 평가 (새로 생성하면서 평가)
    - 주의: 이 함수는 A/B 결과를 새로 생성합니다.
    - 기존 결과를 활용하려면 evaluate_existing_results()를 사용하세요.
    """
    import random
    
    print(f"\n{'='*100}")
    print(f"📊 {group_name} A/B 테스트 + GPT 평가 (새로 생성)")
    print(f"{'='*100}\n")
    
    samples = random.sample(test_cases, min(sample_size, len(test_cases)))
    
    results = []
    wins = {"A": 0, "B": 0, "TIE": 0}
    total_scores = {"A": [], "B": []}
    
    for i, case in enumerate(samples, 1):
        group = case.get("group", "")
        korean_input = case.get("input", "")
        
        print(f"\n[{i}/{len(samples)}] {group}")
        print(f"-" * 80)
        print(f'한글 입력: "{korean_input}"')
        
        # A/B 광고 문구 생성
        ad_copy_a = generate_ad_copy_control_a(korean_input)
        ad_copy_b = generate_ad_copy_experiment_b(korean_input)
        
        print(f'\n대조군 A: "{ad_copy_a}"')
        print(f'실험군 B: "{ad_copy_b}"')
        
        # GPT 평가
        print("\n🔍 GPT 평가 중...")
        evaluation = evaluate_ad_copy_with_gpt(korean_input, ad_copy_a, ad_copy_b, group)
        
        if "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            winner = evaluation.get("winner", "TIE")
            
            print(f"\n📊 평가 결과:")
            print(f"   대조군 A 점수: {eval_a.get('total', 'N/A')}점")
            print(f"   실험군 B 점수: {eval_b.get('total', 'N/A')}점")
            print(f"   🏆 승자: {'대조군 A' if winner == 'A' else '실험군 B' if winner == 'B' else '무승부'}")
            
            wins[winner] += 1
            if eval_a.get('total'):
                total_scores["A"].append(eval_a.get('total'))
            if eval_b.get('total'):
                total_scores["B"].append(eval_b.get('total'))
            
            results.append({
                "group": group,
                "korean_input": korean_input,
                "ad_copy_a": ad_copy_a,
                "ad_copy_b": ad_copy_b,
                "score_a": eval_a.get('total'),
                "score_b": eval_b.get('total'),
                "winner": winner,
                "evaluation": evaluation
            })
        else:
            print(f"   ❌ 평가 오류: {evaluation['error']}")
    
    return results, wins

### 5.3 S등급 GPT 평가 실행

In [14]:
# S등급 GPT 평가 - 섹션 3에서 생성된 s_grade_results 활용
# sample_size=None 으로 설정하면 전체 40개 평가 (API 비용 주의)
# sample_size=10 으로 설정하면 10개만 샘플링하여 평가

s_eval_results, s_wins = evaluate_existing_results(
    s_grade_results,  # 섹션 3에서 생성된 결과 사용
    "S등급 (감성 및 미각 자극)",
    sample_size=None  # 전체 평가 (또는 원하는 샘플 수로 변경)
)


📊 S등급 (감성 및 미각 자극) GPT 평가 (기존 A/B 결과 활용)

📌 전체 60개 평가

[1/60] S-1
--------------------------------------------------------------------------------
한글 입력: "삼겹살 맛집 홍보, 숯불 향 가득한 특선 고기"
대조군 A: "숯불 삼겹살의 향연"
실험군 B: "숯불향 삼겹살 지금 주문!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 80점
   실험군 B 점수: 85점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 명확한 행동 유도를 포함하여 즉각적인 주문을 유도하는 데 효과적입니다. 또한 업종 특화 키워드를 잘 활용하여 고객의 관심을 끌 수 있습니다.

[2/60] S-1
--------------------------------------------------------------------------------
한글 입력: "치킨 전문점, 바삭하고 촉촉한 후라이드 치킨"
대조군 A: "바삭한 유혹, 치킨의 정석"
실험군 B: "바삭 촉촉, 지금 주문!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 82점
   실험군 B 점수: 91점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B 광고 문구는 감각적 표현과 명확한 행동 유도를 통해 즉각적인 반응을 유도하는 데 더 효과적이다.

[3/60] S-1
--------------------------------------------------------------------------------
한글 입력: "피자 가게 신메뉴, 쭉 늘어나는 치즈 피자"
대조군 A: "치즈가 쭉 늘어나는 피자!"
실험군 B: "치즈가 쭉 늘어나는 피자, 지금 주문"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 80점
   실험군 B 점수: 86점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B 광고 문구는 감각적 자극과 함

### 5.4 A등급 GPT 평가 실행

In [15]:
# A등급 GPT 평가 - 섹션 3에서 생성된 a_grade_results 활용
a_eval_results, a_wins = evaluate_existing_results(
    a_grade_results,  # 섹션 3에서 생성된 결과 사용
    "A등급 (비주얼 변화 및 자기관리)",
    sample_size=None  # 전체 평가
)


📊 A등급 (비주얼 변화 및 자기관리) GPT 평가 (기존 A/B 결과 활용)

📌 전체 40개 평가

[1/40] A-1
--------------------------------------------------------------------------------
한글 입력: "미용실 헤어 스타일링, 트렌디한 변신"
대조군 A: "트렌디한 변신, 지금 시작하세요"
실험군 B: "변신의 시작, 맞춤 스타일링 예약하세"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 86점
   실험군 B 점수: 91점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 맞춤 스타일링이라는 표현을 통해 개인 맞춤 서비스와 전문성을 강조하며, 변신의 시작이라는 표현으로 변화에 대한 기대감을 잘 형성합니다.

[2/40] A-1
--------------------------------------------------------------------------------
한글 입력: "피부관리실, 맑고 깨끗한 피부 만들기"
대조군 A: "깨끗한 피부, 당신의 자신감"
실험군 B: "나만의 맞춤 피부 변신!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 84점
   실험군 B 점수: 92점
   🏆 승자: 실험군 B
   📝 이유: 실험군 B 광고 문구는 맞춤형 피부 변신을 강조하여 개인화된 서비스를 제공한다는 점에서 더 매력적입니다. 이는 타겟 고객에게 더 큰 기대감을 줄 수 있습니다.

[3/40] A-1
--------------------------------------------------------------------------------
한글 입력: "피부/비뇨기과, 전문의 피부 진료"
대조군 A: "전문의의 피부 케어"
실험군 B: "전문의와 함께하는 피부 케어"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 86점
   실험군 B 점수: 89점
   🏆 승자: 실험군 B
   📝 이유: 실험

### 5.5 B+C등급 GPT 평가 실행

In [16]:
# B+C등급 GPT 평가 - 섹션 3에서 생성된 bc_grade_results 활용
bc_eval_results, bc_wins = evaluate_existing_results(
    bc_grade_results,  # 섹션 3에서 생성된 결과 사용
    "B+C등급 (공간 체험/교육 + 신뢰 기반 서비스)",
    sample_size=None  # 전체 평가
)


📊 B+C등급 (공간 체험/교육 + 신뢰 기반 서비스) GPT 평가 (기존 A/B 결과 활용)

📌 전체 60개 평가

[1/60] B-1
--------------------------------------------------------------------------------
한글 입력: "입시학원, 명문대 합격의 지름길"
대조군 A: "명문대, 입시의 시작!"
실험군 B: "명문대 합격, 체계적 관리로!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 75점
   실험군 B 점수: 86점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 체계적 관리라는 신뢰 요소를 강조하여 학원의 전문성과 목표 달성 가능성을 더 잘 전달하고 있습니다.

[2/60] B-1
--------------------------------------------------------------------------------
한글 입력: "외국어학원, 글로벌 인재 양성"
대조군 A: "글로벌 인재, 여기서 시작!"
실험군 B: "글로벌 인재, 함께 성장해요"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 84점
   실험군 B 점수: 88점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 '함께 성장'이라는 메시지를 통해 학습자와의 유대감을 강조하며, 지속적인 발전을 기대하게 만듭니다. 이는 학부모와 성인 학습자에게 더 큰 매력을 줄 수 있습니다.

[3/60] B-1
--------------------------------------------------------------------------------
한글 입력: "전문자격 학원, 자격증 취득 전문"
대조군 A: "자격증 취득, 여기서 시작!"
실험군 B: "자격증, 체계적 관리로 취득하세요"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 78점
   실험군 B 점수: 88점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는

### 5.6 D+E등급 GPT 평가 실행

In [17]:
# D+E등급 GPT 평가 - 섹션 3에서 생성된 de_grade_results 활용
de_eval_results, de_wins = evaluate_existing_results(
    de_grade_results,  # 섹션 3에서 생성된 결과 사용
    "D+E등급 (목적형 소매 + 인프라)",
    sample_size=None  # 전체 평가
)


📊 D+E등급 (목적형 소매 + 인프라) GPT 평가 (기존 A/B 결과 활용)

📌 전체 50개 평가

[1/50] D-1
--------------------------------------------------------------------------------
한글 입력: "타이어 매장, 안전한 주행의 시작"
대조군 A: "안전한 주행, 여기서 시작!"
실험군 B: "안전한 주행, 믿고 선택하세요"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 75점
   실험군 B 점수: 81점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 신뢰를 강조하며 고객에게 선택을 유도하는 메시지가 강력하여 더 효과적입니다.

[2/50] D-1
--------------------------------------------------------------------------------
한글 입력: "자동차 부품 매장, 정품 부품 전문"
대조군 A: "정품 부품, 안전한 선택"
실험군 B: "정품 부품, 신뢰의 선택"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 89점
   실험군 B 점수: 91점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 '신뢰의 선택'이라는 표현을 통해 고객에게 더 큰 안정감을 제공하며, 정품 부품의 신뢰성을 더욱 부각합니다.

[3/50] D-1
--------------------------------------------------------------------------------
한글 입력: "모터사이클 부품, 오토바이 액세서리"
대조군 A: "모터사이클의 완벽한 동반자"
실험군 B: "모터사이클, 신나는 여정의 시작!"

🔍 GPT 평가 중...

📊 평가 결과:
   대조군 A 점수: 60점
   실험군 B 점수: 63점
   🏆 승자: 실험군 B
   📝 이유: 광고 문구 B는 모터사이클의 모험적 이미지를 강조하여 감성적인 연결을 유도합니다. 이는 타겟 고객의 

### 5.7 전체 GPT 평가 종합 결과

In [18]:
# 전체 결과 종합
all_eval_results = s_eval_results + a_eval_results + bc_eval_results + de_eval_results
all_wins = {
    "A": s_wins["A"] + a_wins["A"] + bc_wins["A"] + de_wins["A"],
    "B": s_wins["B"] + a_wins["B"] + bc_wins["B"] + de_wins["B"],
    "TIE": s_wins["TIE"] + a_wins["TIE"] + bc_wins["TIE"] + de_wins["TIE"]
}

total_evaluated = len(all_eval_results)

print("\n" + "="*100)
print("🏆 전체 GPT 평가 종합 결과")
print("="*100)

print(f"\n총 평가 케이스: {total_evaluated}개")
print(f"\n📊 전체 승패 현황:")
print(f"   대조군 A 승: {all_wins['A']}회 ({all_wins['A']/total_evaluated*100:.1f}%)")
print(f"   실험군 B 승: {all_wins['B']}회 ({all_wins['B']/total_evaluated*100:.1f}%)")
print(f"   무승부: {all_wins['TIE']}회 ({all_wins['TIE']/total_evaluated*100:.1f}%)")

# 등급별 결과 테이블
print("\n" + "-"*50)
print("등급별 승패 현황")
print("-"*50)
grade_results = pd.DataFrame({
    "등급": ["S등급", "A등급", "B+C등급", "D+E등급", "전체"],
    "대조군 A 승": [s_wins["A"], a_wins["A"], bc_wins["A"], de_wins["A"], all_wins["A"]],
    "실험군 B 승": [s_wins["B"], a_wins["B"], bc_wins["B"], de_wins["B"], all_wins["B"]],
    "무승부": [s_wins["TIE"], a_wins["TIE"], bc_wins["TIE"], de_wins["TIE"], all_wins["TIE"]]
})
print(grade_results.to_string(index=False))

# 평균 점수 계산
all_scores_a = [r["score_a"] for r in all_eval_results if r.get("score_a")]
all_scores_b = [r["score_b"] for r in all_eval_results if r.get("score_b")]

if all_scores_a and all_scores_b:
    print(f"\n📈 전체 평균 점수:")
    print(f"   대조군 A: {sum(all_scores_a)/len(all_scores_a):.1f}점")
    print(f"   실험군 B: {sum(all_scores_b)/len(all_scores_b):.1f}점")
    print(f"   점수 차이: {sum(all_scores_b)/len(all_scores_b) - sum(all_scores_a)/len(all_scores_a):+.1f}점")

# 결과 저장
eval_df = pd.DataFrame([{
    "group": r["group"],
    "korean_input": r["korean_input"],
    "score_a": r["score_a"],
    "score_b": r["score_b"],
    "winner": r["winner"],
    "ad_copy_a": r["ad_copy_a"],
    "ad_copy_b": r["ad_copy_b"]
} for r in all_eval_results])

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
eval_output_path = f"{timestamp}_ab_test_ad_copy_gpt_evaluation.csv"
eval_df.to_csv(eval_output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 GPT 평가 결과 저장 완료: {eval_output_path}")


🏆 전체 GPT 평가 종합 결과

총 평가 케이스: 210개

📊 전체 승패 현황:
   대조군 A 승: 20회 (9.5%)
   실험군 B 승: 184회 (87.6%)
   무승부: 6회 (2.9%)

--------------------------------------------------
등급별 승패 현황
--------------------------------------------------
   등급  대조군 A 승  실험군 B 승  무승부
  S등급        5       55    0
  A등급        5       35    0
B+C등급        6       50    4
D+E등급        4       44    2
   전체       20      184    6

📈 전체 평균 점수:
   대조군 A: 78.2점
   실험군 B: 84.2점
   점수 차이: +5.9점

💾 GPT 평가 결과 저장 완료: 20260127_040826_ab_test_ad_copy_gpt_evaluation.csv


## 7. 결론

### 실험 설계

| 항목 | 대조군 A | 실험군 B |
|------|---------|----------|
| 방식 | 일반 OpenAI API | 업종별 특화 시스템 |
| 광고 문구 구조 | 단순 번역 + 일반 지시문 | 21개 하위 그룹별 최적화 |
| 톤앤매너 | 고정 | 업종별 자동 선택 |
| AIDA 프레임워크 | X | O |
| 타겟 고객 반영 | X | O |
| 업종별 키워드 | X | O (YAML 기반) |

### GPT 평가 기준 (업종별_분류_및_광고전략.md 기반)

| 평가 항목 | 배점 | 설명 |
|----------|------|------|
| 감각/가치 표현 | 20점 | 업종별 핵심 감각적/가치 표현 |
| 타겟 적합성 | 20점 | 타겟 고객에 맞는 메시지 |
| 톤앤매너 | 20점 | 업종에 맞는 브랜드 보이스 |
| CTA/신뢰 | 20점 | 행동 유도 또는 신뢰 요소 |
| 업종 특화 키워드 | 20점 | 업종별 전문 용어 포함 |
| **총점** | **100점** | |

### 기대 효과

1. **업종별 최적화**: 실험군 B는 각 업종의 특성을 반영한 톤앤매너와 키워드를 자동 적용
2. **타겟 맞춤**: 업종별 타겟 고객에 최적화된 메시지 전달
3. **전문성**: 업종별 전문 용어와 광고 문법 활용
4. **객관적 검증**: GPT 기반 자동 평가로 품질 차이를 정량적으로 측정

## 8. B 실험군 패배/무승부 케이스 분석

실험군 B(업종별 특화 시스템)가 대조군 A(일반 OpenAI API)에게 패배하거나 무승부인 케이스를 분석합니다.
이 케이스들은 `industries.yaml` 템플릿 개선의 핵심 데이터가 됩니다.

In [19]:
# ============================================
# B 실험군 패배/무승부 케이스 수집 및 분석
# ============================================

def collect_b_loss_or_tie_cases(all_eval_results: List[Dict]) -> List[Dict]:
    """
    B 실험군이 패배하거나 무승부인 케이스 수집
    
    Args:
        all_eval_results: 전체 GPT 평가 결과 리스트
    
    Returns:
        B가 지거나 비긴 케이스 리스트
    """
    b_loss_or_tie = []
    
    for result in all_eval_results:
        winner = result.get("winner")
        if winner in ["A", "TIE"]:  # A가 이기거나 무승부
            b_loss_or_tie.append(result)
    
    return b_loss_or_tie


def display_b_loss_cases(cases: List[Dict], title: str = "B 실험군 패배/무승부 케이스"):
    """
    B 실험군 패배/무승부 케이스를 상세히 출력
    """
    print(f"\n{'='*120}")
    print(f"📊 {title}")
    print(f"{'='*120}")
    print(f"총 {len(cases)}개 케이스 (A 승리 또는 무승부)\n")
    
    # 승패별 분류
    a_wins = [c for c in cases if c.get("winner") == "A"]
    ties = [c for c in cases if c.get("winner") == "TIE"]
    
    print(f"  - A 승리: {len(a_wins)}개")
    print(f"  - 무승부: {len(ties)}개")
    
    # 그룹별 분류
    group_counts = {}
    for case in cases:
        group = case.get("group", "Unknown")
        group_counts[group] = group_counts.get(group, 0) + 1
    
    print(f"\n📌 그룹별 분포:")
    for group, count in sorted(group_counts.items()):
        print(f"  - {group}: {count}개")
    
    print(f"\n{'='*120}")
    print("상세 케이스 분석")
    print(f"{'='*120}\n")
    
    for i, case in enumerate(cases, 1):
        group = case.get("group", "Unknown")
        korean_input = case.get("korean_input", "N/A")
        ad_copy_a = case.get("ad_copy_a", "N/A")
        ad_copy_b = case.get("ad_copy_b", "N/A")
        score_a = case.get("score_a", "N/A")
        score_b = case.get("score_b", "N/A")
        winner = case.get("winner", "N/A")
        evaluation = case.get("evaluation", {})
        
        winner_label = "🏆 A 승리" if winner == "A" else "🤝 무승부"
        
        print(f"\n[{i}/{len(cases)}] {group} - {winner_label}")
        print(f"-" * 100)
        
        # 입력 및 결과
        print(f"\n📝 한글 입력:")
        print(f"   \"{korean_input}\"")
        
        print(f"\n🅰️ 대조군 A 광고 문구 (점수: {score_a}점):")
        print(f"   \"{ad_copy_a}\"")
        
        print(f"\n🅱️ 실험군 B 광고 문구 (점수: {score_b}점):")
        print(f"   \"{ad_copy_b}\"")
        
        # GPT 평가 상세 내용
        if evaluation and "error" not in evaluation:
            eval_a = evaluation.get("evaluation_a", {})
            eval_b = evaluation.get("evaluation_b", {})
            
            print(f"\n📊 GPT 평가 상세:")
            print(f"   [대조군 A]")
            print(f"   - 감성/가치 표현: {eval_a.get('sensory_or_value', 'N/A')}점")
            print(f"   - 타겟 적합성: {eval_a.get('target_fit', 'N/A')}점")
            print(f"   - 톤앤매너: {eval_a.get('tone_manner', 'N/A')}점")
            print(f"   - CTA/신뢰: {eval_a.get('cta_or_trust', 'N/A')}점")
            print(f"   - 업종 특화: {eval_a.get('industry_specific', 'N/A')}점")
            print(f"   - 강점: {eval_a.get('strengths', 'N/A')}")
            print(f"   - 약점: {eval_a.get('weaknesses', 'N/A')}")
            
            print(f"\n   [실험군 B]")
            print(f"   - 감성/가치 표현: {eval_b.get('sensory_or_value', 'N/A')}점")
            print(f"   - 타겟 적합성: {eval_b.get('target_fit', 'N/A')}점")
            print(f"   - 톤앤매너: {eval_b.get('tone_manner', 'N/A')}점")
            print(f"   - CTA/신뢰: {eval_b.get('cta_or_trust', 'N/A')}점")
            print(f"   - 업종 특화: {eval_b.get('industry_specific', 'N/A')}점")
            print(f"   - 강점: {eval_b.get('strengths', 'N/A')}")
            print(f"   - 약점: {eval_b.get('weaknesses', 'N/A')}")
            
            print(f"\n   📝 승패 이유: {evaluation.get('winner_reason', 'N/A')}")
            print(f"   📝 상세 비교: {evaluation.get('detailed_comparison', 'N/A')}")
        
        print(f"\n{'─'*100}")
    
    return cases


# B 패배/무승부 케이스 수집
b_loss_or_tie_cases = collect_b_loss_or_tie_cases(all_eval_results)

# 상세 출력
display_b_loss_cases(b_loss_or_tie_cases, "광고 문구 B 실험군 패배/무승부 케이스")


📊 광고 문구 B 실험군 패배/무승부 케이스
총 26개 케이스 (A 승리 또는 무승부)

  - A 승리: 20개
  - 무승부: 6개

📌 그룹별 분포:
  - A-2: 2개
  - A-3: 1개
  - A-4: 2개
  - B-1: 3개
  - B-3: 3개
  - C-2: 1개
  - C-3: 3개
  - D-1: 1개
  - D-2: 1개
  - D-3: 2개
  - E-1: 1개
  - E-2: 1개
  - S-1: 1개
  - S-2: 2개
  - S-6: 2개

상세 케이스 분석


[1/26] S-1 - 🏆 A 승리
----------------------------------------------------------------------------------------------------

📝 한글 입력:
   "스테이크 레스토랑, 입에서 녹는 프리미엄 안심"

🅰️ 대조군 A 광고 문구 (점수: 86점):
   "입에서 녹는 프리미엄 스테이크"

🅱️ 실험군 B 광고 문구 (점수: 83점):
   "입에서 녹는 안심 스테이크"

📊 GPT 평가 상세:
   [대조군 A]
   - 감성/가치 표현: 18점
   - 타겟 적합성: 18점
   - 톤앤매너: 17점
   - CTA/신뢰: 15점
   - 업종 특화: 18점
   - 강점: 프리미엄이라는 단어가 고급스러움을 잘 전달하며, '입에서 녹는' 표현이 감각적 자극을 준다.
   - 약점: 구체적인 행동 유도 요소가 부족하다.

   [실험군 B]
   - 감성/가치 표현: 17점
   - 타겟 적합성: 18점
   - 톤앤매너: 16점
   - CTA/신뢰: 15점
   - 업종 특화: 17점
   - 강점: '안심 스테이크'라는 구체적인 메뉴 언급으로 고객에게 명확한 이미지를 제공한다.
   - 약점: 프리미엄이라는 고급스러운 이미지 전달이 부족하다.

   📝 승패 이유: 광고 문구 A는 '프리미엄'이라는 단어를 사용하여 고급스러운 이미지를 더 잘 전달하며, 감각적 표현이 더 강력

[{'group': 'S-1',
  'korean_input': '스테이크 레스토랑, 입에서 녹는 프리미엄 안심',
  'ad_copy_a': '입에서 녹는 프리미엄 스테이크',
  'ad_copy_b': '입에서 녹는 안심 스테이크',
  'len_a': 16,
  'len_b': 14,
  'score_a': 86,
  'score_b': 83,
  'winner': 'A',
  'evaluation': {'grade': 'S등급 - 감성 및 미각 자극',
   'evaluation_a': {'sensory_or_value': 18,
    'target_fit': 18,
    'tone_manner': 17,
    'cta_or_trust': 15,
    'industry_specific': 18,
    'total': 86,
    'strengths': "프리미엄이라는 단어가 고급스러움을 잘 전달하며, '입에서 녹는' 표현이 감각적 자극을 준다.",
    'weaknesses': '구체적인 행동 유도 요소가 부족하다.'},
   'evaluation_b': {'sensory_or_value': 17,
    'target_fit': 18,
    'tone_manner': 16,
    'cta_or_trust': 15,
    'industry_specific': 17,
    'total': 83,
    'strengths': "'안심 스테이크'라는 구체적인 메뉴 언급으로 고객에게 명확한 이미지를 제공한다.",
    'weaknesses': '프리미엄이라는 고급스러운 이미지 전달이 부족하다.'},
   'winner': 'A',
   'winner_reason': "광고 문구 A는 '프리미엄'이라는 단어를 사용하여 고급스러운 이미지를 더 잘 전달하며, 감각적 표현이 더 강력하다.",
   'detailed_comparison': "광고 문구 A는 '프리미엄'이라는 단어로 고급스러움을 강조하여 타겟 고객에게 더 적합하다. 반면, 광고 문

## 9. industries.yaml 개선사항 제안

B 실험군 패배/무승부 케이스를 분석하여 `src/generation/text_generation/config/industries.yaml` 파일의 개선사항을 GPT를 활용해 구체적으로 제안합니다.

In [20]:
# ============================================
# industries.yaml 개선사항 제안 시스템
# ============================================

def generate_improvement_suggestions(
    b_loss_cases: List[Dict],
    yaml_file_path: str = "src/generation/text_generation/config/industries.yaml"
) -> Dict:
    """
    B 패배/무승부 케이스를 분석하여 industries.yaml 개선사항 제안
    
    Args:
        b_loss_cases: B가 지거나 비긴 케이스 리스트
        yaml_file_path: industries.yaml 파일 경로
    
    Returns:
        그룹별 개선사항 딕셔너리
    """
    if not b_loss_cases:
        print("분석할 B 패배/무승부 케이스가 없습니다.")
        return {}
    
    # 그룹별로 케이스 분류
    cases_by_group = {}
    for case in b_loss_cases:
        group = case.get("group", "Unknown")
        if group not in cases_by_group:
            cases_by_group[group] = []
        cases_by_group[group].append(case)
    
    print(f"\n{'='*120}")
    print(f"🔧 industries.yaml 개선사항 제안 (광고 문구)")
    print(f"{'='*120}")
    print(f"대상 파일: {yaml_file_path}")
    print(f"분석 대상: {len(b_loss_cases)}개 케이스 ({len(cases_by_group)}개 그룹)")
    print(f"{'='*120}\n")
    
    all_suggestions = {}
    
    for group, cases in cases_by_group.items():
        print(f"\n{'─'*100}")
        print(f"📌 {group} 그룹 분석 중... ({len(cases)}개 케이스)")
        print(f"{'─'*100}")
        
        # 케이스 요약 생성
        case_summaries = []
        for case in cases:
            summary = {
                "korean_input": case.get("korean_input", ""),
                "ad_copy_a": case.get("ad_copy_a", ""),
                "ad_copy_b": case.get("ad_copy_b", ""),
                "score_a": case.get("score_a"),
                "score_b": case.get("score_b"),
                "winner": case.get("winner"),
                "evaluation": {
                    "a_strengths": case.get("evaluation", {}).get("evaluation_a", {}).get("strengths", ""),
                    "a_weaknesses": case.get("evaluation", {}).get("evaluation_a", {}).get("weaknesses", ""),
                    "b_strengths": case.get("evaluation", {}).get("evaluation_b", {}).get("strengths", ""),
                    "b_weaknesses": case.get("evaluation", {}).get("evaluation_b", {}).get("weaknesses", ""),
                    "winner_reason": case.get("evaluation", {}).get("winner_reason", ""),
                    "detailed_comparison": case.get("evaluation", {}).get("detailed_comparison", "")
                }
            }
            case_summaries.append(summary)
        
        # GPT에게 개선사항 요청
        system_prompt = """당신은 소상공인 광고 문구 생성 시스템의 템플릿 전문가입니다.
A/B 테스트 결과를 분석하여 industries.yaml 파일의 개선사항을 제안합니다.

industries.yaml 파일은 다음과 같은 구조를 가집니다:
```yaml
industries:
  카테고리명:
    tone: "톤앤매너 (warm, professional, friendly 등)"
    keywords: [키워드1, 키워드2, ...]
    cta_style: "CTA 스타일"
    value_proposition: "가치 제안 방식"
    target_emotion: "타겟 감정"
    ad_template: "광고 문구 템플릿"
```

분석 결과를 바탕으로 다음을 제안해주세요:
1. 톤앤매너를 어떻게 개선해야 하는지
2. 어떤 키워드를 추가/수정해야 하는지
3. CTA 스타일을 어떻게 개선해야 하는지
4. 가치 제안 방식을 어떻게 개선해야 하는지
5. 타겟 감정을 어떻게 설정해야 하는지
6. 광고 문구 템플릿을 어떻게 개선해야 하는지

반드시 JSON 형식으로 출력하세요."""

        user_prompt = f"""## 분석 대상 그룹
{group}

## A/B 테스트 결과 (B 실험군이 패배하거나 무승부인 케이스)
{json.dumps(case_summaries, ensure_ascii=False, indent=2)}

## 요청사항
위 케이스들을 분석하여 실험군 B(업종별 특화 시스템)가 대조군 A(일반 시스템)보다 성능이 떨어진 원인을 파악하고,
industries.yaml 파일의 해당 그룹 설정을 어떻게 개선해야 하는지 구체적으로 제안해주세요.

특히:
1. 대조군 A가 더 좋았던 이유 분석
2. 실험군 B의 약점 분석
3. industries.yaml에서 개선해야 할 구체적인 항목과 값

출력 형식:
{{
    "group": "{group}",
    "analysis": {{
        "why_a_won": "A가 더 좋았던 이유",
        "b_weaknesses": "B의 약점 분석",
        "common_patterns": "공통적으로 발견된 패턴"
    }},
    "improvements": {{
        "tone": {{
            "current_issue": "현재 톤앤매너의 문제점",
            "suggestion": "개선된 톤앤매너",
            "reason": "변경 이유"
        }},
        "keywords": {{
            "add": ["추가할 키워드들"],
            "remove": ["제거할 키워드들"],
            "reason": "키워드 변경 이유"
        }},
        "cta_style": {{
            "current_issue": "현재 CTA 스타일의 문제점",
            "suggestion": "개선된 CTA 스타일",
            "reason": "변경 이유"
        }},
        "value_proposition": {{
            "current_issue": "현재 가치 제안의 문제점",
            "suggestion": "개선된 가치 제안 방식",
            "reason": "변경 이유"
        }},
        "target_emotion": {{
            "current_issue": "현재 타겟 감정의 문제점",
            "suggestion": "개선된 타겟 감정",
            "reason": "변경 이유"
        }},
        "ad_template": {{
            "current_issue": "현재 템플릿 문제점",
            "suggestion": "개선된 템플릿 예시",
            "reason": "변경 이유"
        }}
    }},
    "priority": "높음/중간/낮음",
    "summary": "전체 개선사항 요약 (2-3문장)"
}}"""

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.5,
                response_format={"type": "json_object"}
            )
            
            suggestion = json.loads(response.choices[0].message.content)
            all_suggestions[group] = suggestion
            
            # 결과 출력
            print(f"\n✅ {group} 분석 완료")
            print(f"\n📊 분석 결과:")
            print(f"   - A 승리 이유: {suggestion.get('analysis', {}).get('why_a_won', 'N/A')}")
            print(f"   - B 약점: {suggestion.get('analysis', {}).get('b_weaknesses', 'N/A')}")
            print(f"\n🔧 주요 개선사항:")
            
            improvements = suggestion.get('improvements', {})
            
            if improvements.get('tone', {}).get('suggestion'):
                print(f"   - 톤앤매너 개선: {improvements['tone']['suggestion']}")
            if improvements.get('keywords', {}).get('add'):
                print(f"   - 키워드 추가: {improvements['keywords']['add']}")
            if improvements.get('cta_style', {}).get('suggestion'):
                print(f"   - CTA 개선: {improvements['cta_style']['suggestion']}")
            if improvements.get('ad_template', {}).get('suggestion'):
                print(f"   - 템플릿 개선: {improvements['ad_template']['suggestion'][:150]}...")
            
            print(f"\n   📌 우선순위: {suggestion.get('priority', 'N/A')}")
            print(f"   📝 요약: {suggestion.get('summary', 'N/A')}")
            
        except Exception as e:
            print(f"   ❌ 분석 오류: {str(e)}")
            all_suggestions[group] = {"error": str(e)}
        
        # API 레이트 리밋 방지
        import time
        time.sleep(1)
    
    return all_suggestions


# 개선사항 생성
improvement_suggestions = generate_improvement_suggestions(b_loss_or_tie_cases)


🔧 industries.yaml 개선사항 제안 (광고 문구)
대상 파일: src/generation/text_generation/config/industries.yaml
분석 대상: 26개 케이스 (15개 그룹)


────────────────────────────────────────────────────────────────────────────────────────────────────
📌 S-1 그룹 분석 중... (1개 케이스)
────────────────────────────────────────────────────────────────────────────────────────────────────

✅ S-1 분석 완료

📊 분석 결과:
   - A 승리 이유: 광고 문구 A는 '프리미엄'이라는 단어를 사용하여 고급스러운 이미지를 더 잘 전달하며, 감각적 표현이 더 강력하다.
   - B 약점: 광고 문구 B는 구체적인 메뉴 언급으로 명확성을 제공하지만, '프리미엄'이라는 고급스러운 이미지 전달이 부족하다.

🔧 주요 개선사항:
   - 톤앤매너 개선: 고급스럽고 감각적인 톤
   - 키워드 추가: ['프리미엄', '고급', '감각적']
   - CTA 개선: 즉각적인 행동을 유도하는 명령형 스타일
   - 템플릿 개선: 예: '입에서 녹는 프리미엄 스테이크를 경험하세요. 지금 예약하세요!'...

   📌 우선순위: 높음
   📝 요약: S-1 그룹의 광고 문구는 고급스러운 이미지를 강조하고, 감각적 자극을 강화하며, 즉각적인 행동을 유도하는 방향으로 개선되어야 합니다. 이를 통해 고객의 기대를 충족시키고, 더 효과적인 광고 성과를 낼 수 있습니다.

────────────────────────────────────────────────────────────────────────────────────────────────────
📌 S-2 그룹 분석 중... (2개 케이스)
───────────────────────────────────

In [21]:
# ============================================
# 개선사항 종합 보고서 저장
# ============================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"{timestamp}_ab_test_광고문구_개선안.json"

def save_improvement_report(
    suggestions: Dict,
    b_loss_cases: List[Dict],
    output_path: str = f"{timestamp}_ab_test_광고문구_개선안.json"
):
    """
    개선사항 보고서를 JSON 파일로 저장
    """
    report = {
        "report_type": "광고 문구 템플릿 개선사항",
        "target_file": "src/generation/text_generation/config/industries.yaml",
        "analysis_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "total_b_loss_cases": len(b_loss_cases),
        "groups_analyzed": len(suggestions),
        "suggestions_by_group": suggestions,
        "b_loss_cases_detail": b_loss_cases
    }

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ 개선사항 보고서 저장 완료: {output_path}")
    return report


# 보고서 저장
improvement_report = save_improvement_report(
    improvement_suggestions,
    b_loss_or_tie_cases,
    output_path
)

# 종합 요약 출력
print(f"\n{'='*120}")
print("📋 광고 문구 템플릿 개선사항 종합 요약")
print(f"{'='*120}")

priority_high = [g for g, s in improvement_suggestions.items() if s.get('priority') == '높음']
priority_medium = [g for g, s in improvement_suggestions.items() if s.get('priority') == '중간']
priority_low = [g for g, s in improvement_suggestions.items() if s.get('priority') == '낮음']

print(f"\n🔴 높은 우선순위 ({len(priority_high)}개 그룹): {priority_high}")
print(f"🟡 중간 우선순위 ({len(priority_medium)}개 그룹): {priority_medium}")
print(f"🟢 낮은 우선순위 ({len(priority_low)}개 그룹): {priority_low}")

print(f"\n📝 그룹별 핵심 개선사항:")
for group, suggestion in improvement_suggestions.items():
    if "error" not in suggestion:
        print(f"\n  [{group}]")
        print(f"  {suggestion.get('summary', 'N/A')}")


✅ 개선사항 보고서 저장 완료: 20260127_041341_ab_test_광고문구_개선안.json

📋 광고 문구 템플릿 개선사항 종합 요약

🔴 높은 우선순위 (15개 그룹): ['S-1', 'S-2', 'S-6', 'A-2', 'A-3', 'A-4', 'B-1', 'B-3', 'C-2', 'C-3', 'D-1', 'D-2', 'D-3', 'E-1', 'E-2']
🟡 중간 우선순위 (0개 그룹): []
🟢 낮은 우선순위 (0개 그룹): []

📝 그룹별 핵심 개선사항:

  [S-1]
  S-1 그룹의 광고 문구는 고급스러운 이미지를 강조하고, 감각적 자극을 강화하며, 즉각적인 행동을 유도하는 방향으로 개선되어야 합니다. 이를 통해 고객의 기대를 충족시키고, 더 효과적인 광고 성과를 낼 수 있습니다.

  [S-2]
  S-2 그룹의 광고 문구는 감각적 자극과 고급스러움을 강조하고, 명확한 행동 유도를 포함해야 합니다. 이를 통해 고객의 감성을 자극하고 즉각적인 행동을 유도할 수 있습니다.

  [S-6]
  S-6 그룹의 광고 문구는 감성적이고 업종 특화된 표현을 통해 고객의 감성을 자극하고 매력을 전달하는 데 집중해야 합니다. 감성적 톤과 업종 특화 키워드를 활용하여 즉각적인 행동을 유도할 수 있는 광고 문구 개선이 필요합니다.

  [A-2]
  광고 문구의 전문성과 구체성을 강화하여 고객에게 명확한 메시지를 전달해야 합니다. 친근함을 유지하면서도 전문성을 잃지 않는 톤앤매너를 설정하고, 구체적인 변화와 결과를 강조하는 키워드와 가치 제안을 활용해야 합니다.

  [A-3]
  A/B 테스트 결과, 트렌드를 강조하는 광고 문구가 더 효과적이었습니다. 따라서, 톤앤매너, 키워드, CTA 스타일, 가치 제안, 타겟 감정, 광고 템플릿을 최신 패션 트렌드를 강조하는 방향으로 개선해야 합니다.

  [A-4]
  실험군 B의 광고 문구는 감성적 접근에 치우쳐 구체성과 명확성이 부족했습니다. 구체적이고 명확한 메시지를 통해 신뢰감을 주고, 고객이 기대할 수 